In [1]:
import numpy as np
from dynamicSceneGenerator import DynamicSceneGenerator
from visualize import visualize, visualize_camera_gif, visualize_projections, visualize_bounding_boxes
from datatypes.virtualCamera import VirtualCamera

# Might need to pip install json, json_stream, as they are not default libraries
import json
import os

In [2]:
def calculate_camera_position_WRF(focal_length, width_of_sensor, largest_radius, path_centre, camera_height):
    angle_of_view = 2*np.arctan(width_of_sensor/(2*focal_length)) # In radians
    d = largest_radius/np.tan(angle_of_view/2)
    # Assume camera parallel to x axis and always pointing towards the centre of the path circle
    x = d+path_centre[0]
    y = path_centre[1]
    z = camera_height
    return np.array([x,y,z])

def calculate_camera_pitch(camera_position):
    alpha = np.arctan(camera_position[2]/camera_position[0]) # arctan(height/distance)
    return alpha

In [3]:
def create_and_place_simple_legacy_camera(largest_radius, path_centre): # This should maybe get a better name, and should be moved somewhere?
    '''
    Function for placing a Simple legacy photo camera in the dynamic scene
    '''
    # NB! Make sure everything is in meters
    focal_length = 50*10**-3
    image_bounds = (3600, 2400) # Pixels (x,y)
    film_size = (36*10**-3, 24*10**-3)
    px = film_size[0]/image_bounds[0]
    py = film_size[1]/image_bounds[1]
    principal_point = (image_bounds[0]/2,image_bounds[1]/2)
    width_of_sensor = 36*10**-3 # Width of sensor
    camera_height = 60 # metre

    position_WRF = calculate_camera_position_WRF(focal_length, width_of_sensor, largest_radius, path_centre, camera_height)
    roll = np.pi/20
    yaw = np.pi
    pitch = calculate_camera_pitch(position_WRF)
    
    camera = VirtualCamera(position_WRF, roll, yaw, pitch, focal_length, px, py, principal_point, image_bounds)

    return camera

In [4]:
def project_all_points(camera, vessels):
    all_projected_points = {}
    for t in vessels[0].get_track().get_time_stamps():
        points = [vessel.calculate_3D_cornerpoints(t) for vessel in vessels]
        projected_points = [camera.project_points(vessel_points) for vessel_points in points]
        all_projected_points[t] = projected_points
    return all_projected_points

# Where should we save the projected points? A separate class, in the dynamic scene?
# And also the functions in this file should be in a separate class?

In [5]:
def create_bound_boxes(projected_points):
    bbs = {}
    for vesselID, vessel_dict in projected_points.items():
        x_vals = list(map(lambda v : v['x'], vessel_dict.values()))
        y_vals = list(map(lambda v : v['y'], vessel_dict.values()))
        max_x = np.max(x_vals)
        min_x = np.min(x_vals)
        max_y = np.max(y_vals)
        min_y = np.min(y_vals)
        width = max_x-min_x
        height = max_y - min_y
        center = [min_x+width/2, min_y+height/2]
        #bb = np.array([[min_x,min_y],[min_x, max_y],[max_x, max_y],[max_x, min_y]]) 
        bb = {'centre': {'x': center[0], 'y': center[1]}, 'height': height, 'width': width} 
        # Make sure these are in the correct order as in master thesis
        bbs[vesselID] = bb
    return bbs


In [6]:
def create_all_bbs(all_projected_points):
    all_bbs = {}
    for t in all_projected_points.keys():
        all_bbs[t]=create_bound_boxes(all_projected_points[t])
    return all_bbs

In [7]:
def dict_to_json(path, dict):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    with open(path, 'w+') as f:
        json.dump(dict, f, indent = 4)


In [8]:
def tracks_to_json(vessels):
    # NB! Assumes all vessels have the same timestamp
    all_tracks = {key: {vessel.id: vessel.get_track_dict()[key] for vessel in vessels} for key in vessels[0].get_track_dict().keys()}
    arr = os.listdir('./simulations/')
    timestamp = 0 if not arr else int(arr[-1])+1
    filename = f'./simulations/{timestamp}/tracks.json'
    dict_to_json(filename, all_tracks)

In [9]:
# Generate dynamic scene with random tracks
dsg = DynamicSceneGenerator()
dsg.set_random_vessels(6)
dsg.generate_random_tracks()
vessels = dsg.get_vessels()
tracks_to_json(vessels)
# visualize(vessels)

In [10]:
camera = create_and_place_simple_legacy_camera(dsg.get_larges_radius(), dsg.get_path_centre())
# visualize_camera_gif(camera, vessels)

In [11]:
all_projected_points = project_all_points(camera, vessels)
# visualize_projections(all_projected_points, camera.image_bounds)

In [12]:
# All projected points will probably be on a different format. Did this to get a file.
projected_points_dict = {key: {vessels[i].id: {x: {'x': all_projected_points[key][i][x][0], 'y': all_projected_points[key][i][x][1], 'z': all_projected_points[key][i][x][2]} for x in range(len(all_projected_points[key][i]))} for i in range(len(all_projected_points[key]))} for key in all_projected_points.keys()}
filename = f'./simulations/0/projectedPoints.json'
dict_to_json(filename, projected_points_dict)

In [15]:
all_bbs = create_all_bbs(projected_points_dict)
filename = f'./simulations/0/boundingBoxes.json'
dict_to_json(filename, all_bbs)

# With points in BBs
visualize_bounding_boxes(all_bbs, camera.image_bounds, projected_points=projected_points_dict, show_projected_points=True)
# Without
#visualize_bounding_boxes(all_bbs, camera.image_bounds)


In [49]:
all_bbs[0]

{'vessel0': {'centre': [2624.474646311678, 1426.415734021733],
  'height': 224.94932676309872,
  'width': 90.47097507569742},
 'vessel1': {'centre': [911.4188280344054, 1087.3509865672036],
  'height': 38.92808796325062,
  'width': 64.38445007618543},
 'vessel2': {'centre': [2647.9428295021403, 1456.245269432893],
  'height': 51.80658335462863,
  'width': 60.92677726126658},
 'vessel3': {'centre': [2136.5624365201575, 1355.7729371241148],
  'height': 218.85866726304903,
  'width': 155.9887533458732},
 'vessel4': {'centre': [2600.6073507978085, 1426.4620209883938],
  'height': 49.65448359449124,
  'width': 66.32634902290283},
 'vessel5': {'centre': [1991.3573576602466, 1349.3025596772604],
  'height': 186.47324352368378,
  'width': 432.5462304735818}}

![alt text](gifs/dynamicSceneExample.gif "Dynamic scene")

![alt text](gifs/camera_position.gif "Camera scene")


![alt text](gifs/projected_points.gif "Projected scene")


![alt text](gifs/boundingBoxes.gif "Projected scene")
